In [2]:
import numpy as np
import pandas as pd

In [ ]:
class LogisticRegressor:
    def __init__(self, alpha=0.01, max_iters=100000, threshold=1e-6, N=1000, hidden_layers=1, hidden_layer_size=3):
        self.alpha = alpha
        self.max_iters = max_iters
        self.threshold = threshold
        self.N = N
        self.hidden_layers = hidden_layers
        self.hidden_layer_size = hidden_layer_size # not sure what to do with this. 
        self.trained = False #to be used in predict
        


    def sigmoid(self, t):
        return 1/(1 + np.exp(-t))
    
    def loss_function(self, y, y_cap):
        epsilon = np.finfo(float).eps
        y_cap = np.clip(y_cap, epsilon, 1 - epsilon)
        return -(y * np.log(y_cap) + (1 - y) * np.log(1 - y_cap))
    
    def fit(self, X, Y):

        
        m = X.shape[0] # samples
        n = X.shape[1] # features

        #init weights and bias
        W1 = np.random.randn(self.hidden_layer_size,n) * 0.01 #random values with mean=0, stdev=0.01 ?
        b1 = np.zeros((self.hidden_layer_size,1))

        W2 = np. random.randn(1,self.hidden_layer_size) * 0.01 
        b2 = np.zeros((1, 1))

        X = np.array(X)
        Y = np.array(Y).reshape(-1, 1)


        stopping = False; loss_prev = 0; iteration = 0; acc = 0
        

        while not stopping:
            #select random data point
            
            i = np.random.choice(np.arange(m))  # pick a random index
            x = X[i].reshape(-1,1)
            y = Y[i]

            #forward pass
            a1  = self.sigmoid(W1 @ x + b1) 
            y_cap = self.sigmoid(W2 @ a1 + b2)
            
            
            loss = self.loss_function(y, y_cap)
            
            #back propagation
            #output layer
            deltaz = y_cap - y
            delW2 = deltaz @ a1.T
            delb2 = deltaz
            #hidden layer
            deltaz1  = (W2.T @ deltaz) * (a1 * (1 - a1))
            delW1 = deltaz1 @ x.T    # gradient of hidden weights
            delb1 = deltaz1           # gradient of hidden biases

            #stochatic gradient descent
            W1 = W1 - self.alpha * delW1
            b1 = b1 - self.alpha * delb1
            W2 = W2 - self.alpha * delW2
            b2 = b2 - self.alpha * delb2

            if abs(loss - loss_prev) < self.threshold:
                stopping = True
            loss_prev = loss
            iteration += 1
            if iteration >= self.max_iters:
                stopping = True

        #set weights here
        self.W1 = W1
        self.b1 = b1
        self.W2 = W2
        self.b2 = b2
        self.trained = True

    def predict(self,x):
        if (self.trained == False):
            raise ValueError("Not trained uyet!")
        
        #change here
        a1 = self.sigmoid(self.W1 @ x.T + self.b1)
        y_prob = self.sigmoid(self.W2 @ a1 + self.b2)

        Y_pred = (y_prob >= 0.5).astype(int)
        return Y_pred
    
    def score(self, x_test, y_test):
        if (self.trained == False):
            raise ValueError("Not trained uyet!")        
        
        y_pred = self.predict(x_test)
        #running into shape issues so trying this 
        y_pred = y_pred.flatten()
        y_test = y_test.flatten()
        correct = 0
        size = len(y_test)

        for i in range(size):
            if (y_pred[i]==y_test[i]):
                correct+=1

        return correct/size
    

In [13]:

#file_name = r"../datasets/blobs600.csv"
file_name = r"../datasets/circles500.csv"
df1 = pd.read_csv(file_name)
Y = df1['Class'].values
del df1['Class']  
X = df1.values     
X.shape

(500, 2)